In [1]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base'}

In [2]:
from pyqir import Module, Context, Call, qubit_id

def print_circuit(program):
    # helper function for adding a single qubit gate to a wire
    def add_gate(gate, wire):
        wires[wire].append(gate)

    # helper function for adding a two qubit gate to a wire
    def add_controlled_gate(gate, control_wire, target_wire):
        add_two_qubit_gate("●-", gate, control_wire, target_wire)

    # helper function for adding a two-qubit gate to a wire
    def add_two_qubit_gate(gate1, gate2, wire1, wire2):
        # get the longest length of any wire
        min_wire = min(wire1, wire2)
        max_wire = max(wire1, wire2)
        max_len = max([len(wire) for wire in wires[min_wire:max_wire + 1]])
        # pad the shorter wires with lines
        for wire in wires[min_wire:max_wire + 1]:
            if len(wire) < max_len:
                wire += ["--"] * (max_len - len(wire))
        # add the gates to the wires
        add_gate(gate1, wire1)
        add_gate(gate2, wire2)
        # add a line connecting the two wires
        for i in range(min_wire + 1, max_wire):
            wires[i].append("|-")

    # helper function for doubly-controlled gates
    def add_doubly_controlled_gate(gate, control_wire1, control_wire2, target_wire):
        # get the longest length of any wire
        min_wire = min(control_wire1, control_wire2, target_wire)
        max_wire = max(control_wire1, control_wire2, target_wire)
        max_len = max([len(wire) for wire in wires[min_wire:max_wire + 1]])
        # pad the shorter wires with lines
        for wire in wires[min_wire:max_wire + 1]:
            if len(wire) < max_len:
                wire += ["--"] * (max_len - len(wire))
        # add the gates to the wires
        add_gate(gate, target_wire)
        add_gate("●-", control_wire1)
        add_gate("●-", control_wire2)
        # add a line connecting the two wires
        for i in range(min_wire + 1, max_wire):
            if len(wires[i]) <= max_len:
                wires[i].append("|-")

    mod = Module.from_ir(Context(), str(program))
    func = mod.functions[0]
    num_qubits = int(func.attributes.func["required_num_qubits"].string_value)

    # Create an array of empty arrays for each qubit
    wires = [[] for _ in range(num_qubits)]

    for instr in func.basic_blocks[0].instructions:
        if isinstance(instr, Call):
            if instr.callee.name == "__quantum__qis__h__body":
                add_gate("H-", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__s__body":
                add_gate("S-", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__s__adj":
                add_gate("S†", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__t__body":
                add_gate("T-", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__t__adj":
                add_gate("T†", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__x__body":
                add_gate("X-", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__y__body":
                add_gate("Y-", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__z__body":
                add_gate("Z-", qubit_id(instr.args[0]))
            elif instr.callee.name == "__quantum__qis__rx__body":
                add_gate("Rx", qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__ry__body":
                add_gate("Ry", qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__rz__body":
                add_gate("Rz", qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__rxx__body":
                add_two_qubit_gate("Rx", "Rx", qubit_id(instr.args[1]), qubit_id(instr.args[2]))
            elif instr.callee.name == "__quantum__qis__ryy__body":
                add_two_qubit_gate("Ry", "Ry", qubit_id(instr.args[1]), qubit_id(instr.args[2]))
            elif instr.callee.name == "__quantum__qis__rzz__body":
                add_two_qubit_gate("Rz", "Rz", qubit_id(instr.args[1]), qubit_id(instr.args[2]))
            elif instr.callee.name == "__quantum__qis__cx__body":
                add_controlled_gate("X-", qubit_id(instr.args[0]), qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__cy__body":
                add_controlled_gate("Y-", qubit_id(instr.args[0]), qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__cz__body":
                add_controlled_gate("Z-", qubit_id(instr.args[0]), qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__swap__body":
                add_two_qubit_gate("X-", "X-", qubit_id(instr.args[0]), qubit_id(instr.args[1]))
            elif instr.callee.name == "__quantum__qis__ccx__body":
                add_doubly_controlled_gate("X-", qubit_id(instr.args[0]), qubit_id(instr.args[1]), qubit_id(instr.args[2]))

    # padd the wires up to the longest length
    max_len = max([len(wire) for wire in wires])
    for wire in wires:
        if len(wire) < max_len:
            wire += ["--"] * (max_len - len(wire))

    for (i, wire) in enumerate(wires):
        print("q" + str(i) + ":\t" + "".join(wire))

In [3]:
%%qsharp
open Microsoft.Quantum.Measurement;
operation GHZ(num : Int) : Result[] {
    use qs = Qubit[1+ num];
    H(qs[0]);
    for q in qs[1...] {
        CNOT(qs[0], q);
    }
    return MResetEachZ(qs);
}

In [4]:
print_circuit(qsharp.compile("GHZ(3)"))

q0:	H-●-●-●-
q1:	--X-|-|-
q2:	----X-|-
q3:	------X-


In [5]:
%%qsharp
/// # Sample
/// Grover's search algorithm
///
/// # Description
/// Grover's search algorithm is a quantum algorithm that finds with high
/// probability the unique input to a black box function that produces a
/// particular output value.
///
/// This Q# program implements the Grover's search algorithm.
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Diagnostics;

operation Grover(nQubits : Int) : Result[] {
    // Grover's algorithm relies on performing a "Grover iteration" an
    // optimal number of times to maximize the probability of finding the
    // value we are searching for.
    // You can set the number iterations to a value lower than optimal to
    // intentionally reduce precision.
    let iterations = CalculateOptimalIterations(nQubits);
    Message($"Number of iterations: {iterations}");

    // Use Grover's algorithm to find a particular marked state.
    let results = GroverSearch(nQubits, iterations, ReflectAboutMarked);
    return results;
}

/// # Summary
/// Implements Grover's algorithm, which searches all possible inputs to an
/// operation to find a particular marked state.
operation GroverSearch(
    nQubits : Int,
    iterations : Int,
    phaseOracle : Qubit[] => Unit) : Result[] {

    use qubits = Qubit[nQubits];

    // Initialize a uniform superposition over all possible inputs.
    PrepareUniform(qubits);

    // The search itself consists of repeatedly reflecting about the marked
    // state and our start state, which we can write out in Q# as a for loop.
    for _ in 1..iterations {
        phaseOracle(qubits);
        ReflectAboutUniform(qubits);
    }

    // Measure and return the answer.
    return MResetEachZ(qubits);
}

/// # Summary
/// Returns the optimal number of Grover iterations needed to find a marked
/// item, given the number of qubits in a register.
function CalculateOptimalIterations(nQubits : Int) : Int {
    if nQubits > 63 {
        fail "This sample supports at most 63 qubits.";
    }
    let nItems = 1 <<< nQubits; // 2^nQubits
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let iterations = Round(0.25 * PI() / angle - 0.5);
    return iterations;
}

/// # Summary
/// Reflects about the basis state marked by alternating zeros and ones.
/// This operation defines what input we are trying to find in the search.
operation ReflectAboutMarked(inputQubits : Qubit[]) : Unit {
    Message("Reflecting about marked state...");
    use outputQubit = Qubit();
    within {
        // We initialize the outputQubit to (|0⟩ - |1⟩) / √2, so that
        // toggling it results in a (-1) phase.
        X(outputQubit);
        H(outputQubit);
        // Flip the outputQubit for marked states.
        // Here, we get the state with alternating 0s and 1s by using the X
        // operation on every other qubit.
        for q in inputQubits[...2...] {
            X(q);
        }
    } apply {
        Controlled X(inputQubits, outputQubit);
    }
}

/// # Summary
/// Given a register in the all-zeros state, prepares a uniform
/// superposition over all basis states.
operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    for q in inputQubits {
        H(q);
    }
}

/// # Summary
/// Reflects about the all-ones state.
operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
    Controlled Z(Most(inputQubits), Tail(inputQubits));
}

/// # Summary
/// Reflects about the uniform superposition state.
operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
    within {
        // Transform the uniform superposition to all-zero.
        Adjoint PrepareUniform(inputQubits);
        // Transform the all-zero state to all-ones
        for q in inputQubits {
            X(q);
        }
    } apply {
        // Now that we've transformed the uniform superposition to the
        // all-ones state, reflect about the all-ones state, then let the
        // within/apply block transform us back.
        ReflectAboutAllOnes(inputQubits);
    }
}

In [6]:
print_circuit(qsharp.compile("Grover(8)"))

q0:	H-X-X-●-T†X-●-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------●-X-T-●-X-X-H-X-X-●-T†X-●-----------------------------------------------------------------------------------------------------------------------------------------------●-X-T-●-X-X-H-X-X-●-T†X-●-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------●-X-T-●-X-X-H-X-X-●-T†X-●-----------------------------------------------------------------------------------------------------------------------------------------------●-X-T-●-X-X-H-X-X-●-T†X-●-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------●-X-T-●-X-X-H-X-X-●-T†X-●-----------------------------------------------

In [7]:
print_circuit(qsharp.compile(
    "{\
        use (q1, q2, q3) = (Qubit(), Qubit(), Qubit());\
        CCNOT(q1, q2, q3);\
        CX(q1, q3);\
        CY(q1, q3);\
        CZ(q1, q3);\
        Rx(0.0, q1);\
        Rxx(0.0, q1, q3);\
        Ry(0.0, q1);\
        Ryy(0.0, q1, q3);\
        Rz(0.0, q1);\
        Rzz(0.0, q1, q3);\
        H(q1);\
        S(q1);\
        Adjoint S(q1);\
        T(q1);\
        Adjoint T(q1);\
        X(q1);\
        Y(q1);\
        Z(q1);\
        SWAP(q1, q3);\
        Reset(q1);\
        (M(q1),\
        Microsoft.Quantum.Measurement.MResetZ(q1))\
        }"
))

q0:	●-●-●-●-RxRxRyRyRzRzH-S-S†T-T†X-Y-Z-X-
q1:	●-|-|-|---|---|---|-----------------|-
q2:	X-X-Y-Z---Rx--Ry--Rz----------------X-
q3:	H-●-H---------------------------------
q4:	--Z-----------------------------------
